In [3]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers
import numpy as np

# Set the path to the extracted training data
training_dir = 'training_set/'

# Define image size and batch size
image_size = (128, 128)
batch_size = 32

# Load training and validation datasets directly from folders
train_ds = image_dataset_from_directory(
    training_dir,
    image_size=image_size,
    batch_size=batch_size,
    labels='inferred',
    label_mode='binary',
    validation_split=0.2,
    subset="training",
    seed=123
)

val_ds = image_dataset_from_directory(
    training_dir,
    image_size=image_size,
    batch_size=batch_size,
    labels='inferred',
    label_mode='binary',
    validation_split=0.2,
    subset="validation",
    seed=123
)

# Normalize pixel values from [0, 255] to [0, 1]
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label

train_ds = train_ds.map(normalize_img)
val_ds = val_ds.map(normalize_img)

print("Image data loaded and preprocessed.")


# Build a simple Convolutional Neural Network (CNN) model
model = tf.keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Sigmoid for binary classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

print("\nModel training complete.")


from sklearn.metrics import accuracy_score, f1_score

# Evaluate the model
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

# Get all true labels and predictions to calculate the F1-score
all_labels = np.concatenate([y for x, y in val_ds], axis=0)
all_predictions = model.predict(val_ds)
predicted_classes = (all_predictions > 0.5).astype("int32").flatten()

# Calculate the weighted F1-score
f1 = f1_score(all_labels, predicted_classes, average='weighted')

print(f"\nModel F1-Score: {f1:.4f}")

NotFoundError: Could not find directory training_set/